In [1]:
from datasets import load_dataset
import pandas as pd

# Step 3: Load the full phishing dataset from Hugging Face
dataset = load_dataset("renemel/compiled-phishing-dataset", split="train")

# Step 4: Convert to pandas DataFrame
df = dataset.to_pandas()

# Step 5: Preview the column names to identify date column
print("Columns available:")
print(df.columns)

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Columns available:
Index(['text', 'type', 'source', 'word_count', 'sentence_count',
       'words_per_sentence', 'domain'],
      dtype='object')


In [2]:
df

,text,type,source,word_count,sentence_count,words_per_sentence,domain
0,"\nHi, Ms Nadia Jules is my name, An Orphan fro...",phishing,deduplicated_dataset.csv,47,3,15.666667,unknown
1,"I am Edward Thomas , There is this client o...",phishing,deduplicated_dataset.csv,101,5,20.200000,fm.bb
2,"My name is Gerald L. Marcus, I know a single b...",phishing,deduplicated_dataset.csv,95,8,11.875000,gmail.com
3,"\nMy Name is Rose Modepe an orphan, I gave bir...",phishing,deduplicated_dataset.csv,56,3,18.666667,unknown
4,\nHi\n\nMy name is Mrs Yetunde Owolabi from Re...,phishing,deduplicated_dataset.csv,90,4,22.500000,unknown
...,...,...,...,...,...,...,...
119143,"fyi , kim .\n- - - - - original message - - - ...",not phishing,enron_spam_data.csv,325,14,23.214286,unknown
119144,"fyi , kim .\n- - - - - original message - - - ...",not phishing,enron_spam_data.csv,325,14,23.214286,unknown
119145,"fyi , kim .\n- - - - - original message - - - ...",not phishing,enron_spam_data.csv,325,14,23.214286,unknown
119146,"fyi , kim .\n- - - - - original message - - - ...",not phishing,enron_spam_data.csv,325,14,23.214286,unknown


-----------------



Types of error rates? Good emails 

Balanced?

- How to quantify?
- Other metrics

Make sure to separate, head-to-head comparison is not right

Baseline vs BERT
(standard tokenization)

Experiment BERT vs BERT (tokens)

Experiment (Train and test(replace 1:1 adjusted values)) - Solutions (can be train the data on those too)
1. Noise - Vecotors of attacks (Is there a standard way of doing so?) Type and how much 
2. Adverserial - what it takes to make it robust agains those type of attacks



-----------------


In [7]:
# Import libraries
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Step 4: Use 'text' as input and 'type' as label
df = df[["text", "type"]].dropna()

# Step 5: Convert 'type' column to binary label (1 = phishing, 0 = legitimate)
df["label"] = df["type"].apply(lambda x: 1 if x.lower() == "phishing" else 0)

# Step 6: TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)
X = tfidf.fit_transform(df["text"])
y = df["label"]

# Step 7: Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 8: Train Logistic Regression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Step 9: Predict and evaluate
y_pred = clf.predict(X_test)

# Step 10: Output results
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
import numpy as np

# 1. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("🔍 Confusion Matrix:")
print(f"True Negatives:  {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives:  {tp}")
print()

# 2. Detailed Classification Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("📊 Model Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")
print()

# 3. Class Distribution in Test Set
unique, counts = np.unique(y_test, return_counts=True)
print("📌 Class Distribution in Test Set:")
for cls, count in zip(unique, counts):
    label = "Legitimate (0)" if cls == 0 else "Phishing (1)"
    print(f"{label}: {count} samples")
print()

# 4. Baseline Accuracy (predicting the majority class)
majority_class_count = max(counts)
baseline_accuracy = majority_class_count / sum(counts)
print(f"⚠️ Baseline Accuracy (majority class only): {baseline_accuracy:.4f}")

🔍 Confusion Matrix:
True Negatives:  9891
False Positives: 169
False Negatives: 234
True Positives:  13536

📊 Model Performance:
Accuracy : 0.9831
Precision: 0.9877
Recall   : 0.9830
F1 Score : 0.9853

📌 Class Distribution in Test Set:
Legitimate (0): 10060 samples
Phishing (1): 13770 samples

⚠️ Baseline Accuracy (majority class only): 0.5778


In [ ]:
from datasets import load_dataset, Dataset
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# Step 2: Split into train and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
import torch

# Step 3: Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Step 4: Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


# Step 5: Tokenize text
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)


train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Step 6: Load pre-trained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


# Step 7: Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


# Step 8: Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Step 9: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Step 10: Train the model
trainer.train()

# Step 11: Evaluate on test set
metrics = trainer.evaluate()
print("📊 Evaluation metrics on test set:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

Map: 100%|██████████| 23830/23830 [00:04<00:00, 5094.78 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import torch

print(torch.__version__)

2.6.0
